In [ ]:
import os
os.environ['WANDB_DISABLED']="true"

# Install and load libraries

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch>

In [ ]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

from datasets import Dataset, load_metric

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
import pandas as pd
import numpy as np

# Paths to the files
path_train_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_NER_Bio/train_short_bio_NER.csv'
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_NER_Bio/test_short_bio_NER.csv'
path_val_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_NER_Bio/val_short_bio_NER.csv'

# Load the datasets
train_short = pd.read_csv(path_train_short)
test_short = pd.read_csv(path_test_short)
val_short = pd.read_csv(path_val_short)

# Check the first few rows to ensure they're loaded correctly
print("Short_dialogue_train_data:")
print(train_short.head())

# Function to calculate statistics
def calculate_statistics(data, text_column, label_column):
    dialogue_lengths = data[text_column].str.split().apply(len)
    label_lengths = data[label_column].str.split().apply(len)

    dialogue_stats = {
        'Min dialogue length': dialogue_lengths.min(),
        'Ave dialogue length': dialogue_lengths.mean(),
        'Max dialogue length': dialogue_lengths.max(),
        '95% dialogue length': np.percentile(dialogue_lengths, 95),
        'Median dialogue length': dialogue_lengths.median()
    }

    label_stats = {
        'Min label length': label_lengths.min(),
        'Ave label length': label_lengths.mean(),
        'Max label length': label_lengths.max(),
        '95% label length': np.percentile(label_lengths, 95),
        'Median label length': label_lengths.median()
    }

    return dialogue_stats, label_stats

# Replace 'dialogue_column' and 'label_column' with the actual column names
dialogue_column = 'dialogue'  # Replace with your actual dialogue column name
label_column = 'bio_ner_unique_no_label'  # Replace with your actual label column name

# Calculate statistics for training data
train_dialogue_stats, train_label_stats = calculate_statistics(train_short, dialogue_column, label_column)
test_dialogue_stats, test_label_stats = calculate_statistics(test_short, dialogue_column, label_column)
val_dialogue_stats, val_label_stats = calculate_statistics(val_short, dialogue_column, label_column)

# Print the results
print("Training Data Dialogue Stats:")
print(train_dialogue_stats)
print("Training Data Label Stats:")
print(train_label_stats)

print("Test Data Dialogue Stats:")
print(test_dialogue_stats)
print("Test Data Label Stats:")
print(test_label_stats)

print("Validation Data Dialogue Stats:")
print(val_dialogue_stats)
print("Validation Data Label Stats:")
print(val_label_stats)


Short_dialogue_train_data:
  section_header                                       section_text  \
0          GENHX  The patient is a 75-year-old female who comes ...   
1      FAM/SOCHX         Significant for diabetes and hypertension.   
2  PASTMEDICALHX                  Significant for anxiety disorder.   
3          GENHX  The patient is a 77-year-old female who is una...   
4      FAM/SOCHX                                   Noncontributory.   

                                            dialogue  \
0  Doctor: Welcome to the clinic. I am Doctor Fra...   
1  Doctor: Does anyone else in your family suffer...   
2  Doctor: Have we gone over your survey results ...   
3  Guest_clinician: How old is the patient? Docto...   
4  Doctor: Do you have a known- Patient: Drug all...   

                                       bio_ner_label  \
0  [{'token': 'stroke', 'label': 'DISEASE'}, {'to...   
1  [{'token': 'high blood pressure', 'label': 'DI...   
2  [{'token': 'anxiety disorder', 'label'

# Load Flan-T5-base

Reference Flan-T5-base [link text](https://https://arxiv.org/pdf/2210.11416)


Based on data analysis
- Min dialogue length: 6
- Ave dialogue length: 104
- Max dialogue length: 1509
- 95% dialogue length: 325
- Median dialogue length: 63
- Min summary length: 1
- Ave summary length: 13
- Max summary length: 82
- 95% summary length: 46
- Median summary length: 9

# Experiment with original model

In [ ]:
model_name='google/flan-t5-base'

t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(t5_model))


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [ ]:
train_short = Dataset.from_pandas(train_short)
test_short = Dataset.from_pandas(test_short)
val_short = Dataset.from_pandas(val_short)


In [ ]:
## test the model

index = 10

dialogue = test_short[index]['dialogue']
summary = test_short[index]['section_text']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    t5_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

Doctor: History of any illnesses, surgeries, or hospitalizations? Patient: I dislocated my wrist when I was a kid but that's all.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Surgeries: None. Injuries: Dislocated wrist. Illnesses: None.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The doctor will take a blood test.


# Fine-tuning the model

## Tokenization

In [ ]:
# tokenize
max_input_length = 1024
max_output_length = 512
def tokenize_function(batch):
    start_prompt = (
    "Summarize the following patient-doctor. "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = '\n\nSummary: '

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + "\n\n" + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['bio_ner_unique_no_label'], batch['dialogue'])]

    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["section_text"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# map the function
batch_size = 2
train_short = train_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text","dialogue","bio_ner_label", "bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/1360 [00:00<?, ? examples/s]

In [ ]:
test_short = test_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text","dialogue","bio_ner_label", "bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [ ]:
val_short = val_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text","dialogue","bio_ner_label", "bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

In [ ]:
from random import sample
# check the shape after tokenization
sample_input_id = test_short['input_ids'][0:1]
sample_label = test_short['labels'][0:1]
sample_attention_mask = test_short['attention_mask'][0:1]
sample_global_attention_mask = test_short['global_attention_mask'][0:1]

print("tokenized_train_exp: ", test_short)
print("\nsample_input_id: ", len(sample_input_id[0]), sample_input_id)
print("\nsample_label: ", len(sample_label[0]), sample_label)
print("\nsample_attention_mask: ", len(sample_attention_mask[0]), sample_attention_mask)
print("\nsample_global_attention_mask: ", len(sample_global_attention_mask[0]), sample_global_attention_mask)


tokenized_train_exp:  Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 222
})

sample_input_id:  1024 [[12198, 1635, 1737, 8, 826, 1868, 18, 26, 32, 5317, 5, 15746, 66, 1035, 120, 2193, 251, 6, 379, 384, 892, 6, 8209, 6, 657, 1035, 11, 11685, 892, 6, 256, 51, 202, 1707, 7, 6, 7690, 772, 6, 11, 801, 18500, 5, 2048, 8, 826, 1035, 11, 5368, 12311, 21527, 45, 8, 7478, 12, 199, 4505, 1635, 1707, 6, 68, 103, 59, 147, 120, 169, 135, 5, 4443, 2197, 10, 784, 31, 13974, 533, 31, 6, 3, 31, 138, 509, 2831, 31, 908, 5267, 10384, 10, 7582, 10, 2018, 6, 132, 5, 17656, 10, 2018, 5, 14252, 834, 15474, 10, 2018, 55, 94, 19, 1245, 12, 942, 25, 5, 7582, 10, 94, 19, 1245, 12, 942, 25, 38, 168, 5, 148, 33, 8, 2553, 6, 2024, 58, 14252, 834, 15474, 10, 2163, 5, 27, 183, 8, 2553, 5, 7582, 10, 571, 307, 43, 25, 192, 118, 4464, 58, 14252, 834, 15474, 10, 101, 43, 118, 4464, 21, 19662, 2641, 203, 5, 7582, 10, 531, 25, 619, 16, 8, 337, 5699, 58, 17656, 10, 2

In [ ]:
decode_text = tokenizer.decode(sample_input_id[0], skip_special_tokens=True)
decode_text

"Summarize the following patient-doctor. Include all medically relevant information, including family history, diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. Entities: ['mental health', 'alcohol'] Dialogue: Doctor: Hi, there. Patient: Hi. Guest_family: Hi! It is nice to meet you. Doctor: It is nice to meet you as well. You are the husband, correct? Guest_family: Yes. I am the husband. Doctor: How long have you two been married? Guest_family: We have been married for forty eight years. Doctor: Do you live in the same household? Patient: Yes. Doctor: Do you have a primary care physician? Patient: No, I am currently looking for someone. I have also been looking for someone to see for my mental health issues. Doctor: I have a few recommendations I can give to you if you would like. Patient: I would like your recommend

## Training

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from transformers import DataCollatorForSeq2Seq

# ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=t5_model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.optim import AdamW

# Clear any cached memory to prevent out-of-memory errors
torch.cuda.empty_cache()

# Data collator is set for dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=t5_model, return_tensors="pt")

# Define custom optimizer
optimizer = AdamW(t5_model.parameters(), lr=1e-4, weight_decay=0.01) # Baseline learning rate was 1e-3

repository_id = 'hankym/flan-t5'  # Hugging Face Hub repository


# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=1e-4,
    num_train_epochs=15,
    weight_decay=0.01,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=t5_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_short,
    eval_dataset=val_short,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),  # Pass the custom optimizer
)

Rouge Scores and Len Gen are normalized in the table elow: it means that what is displyed ust be divided for 100.

As it is shown 24.3183, it is 0.243183.

In [ ]:
# # Start training
torch.cuda.empty_cache()
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.315192,24.318300,8.965100,20.569400,22.148000,14.831933
2,No log,2.275276,25.357500,9.057800,21.632500,23.171500,14.184874
3,2.391900,2.249146,26.497700,9.715700,22.431500,24.017900,14.621849
4,2.391900,2.226628,26.645100,10.677100,22.601000,24.133200,14.831933
5,2.391900,2.216124,26.207400,10.044000,22.093700,23.651400,15.142857
6,2.218500,2.201943,26.629300,10.266000,22.804300,24.432500,15.151261
7,2.218500,2.198858,25.966800,9.341700,22.070300,23.663200,15.092437
8,2.218500,2.188354,26.013400,10.007300,22.411400,23.962100,15.159664
9,2.177200,2.186778,25.325400,9.099800,21.574800,23.186000,15.151261
10,2.177200,2.183101,25.536700,9.364800,21.860500,23.359800,15.109244


TrainOutput(global_step=2550, training_loss=2.2142432598039217, metrics={'train_runtime': 1053.8067, 'train_samples_per_second': 19.358, 'train_steps_per_second': 2.42, 'total_flos': 2.79380999798784e+16, 'train_loss': 2.2142432598039217, 'epoch': 15.0})

In [ ]:
# # save the model
trainer.save_model("./flan_t5_base_finetuned_MTS_bio_unique_dialogue")
tokenizer.save_pretrained("./flan_t5_base_finetuned_MTS_bio_unique_dialogue")

('./flan_t5_base_finetuned_MTS_bio_unique_dialogue/tokenizer_config.json',
 './flan_t5_base_finetuned_MTS_bio_unique_dialogue/special_tokens_map.json',
 './flan_t5_base_finetuned_MTS_bio_unique_dialogue/spiece.model',
 './flan_t5_base_finetuned_MTS_bio_unique_dialogue/added_tokens.json',
 './flan_t5_base_finetuned_MTS_bio_unique_dialogue/tokenizer.json')

In [ ]:
# push to huggingface
from huggingface_hub import HfApi, HfFolder

model_name = "hankym/flan_t5_base_finetuned_MTS_bio_unique_dialogue"
api = HfApi()

# Create the repository if it doesn't exist
api.create_repo(repo_id=model_name, repo_type="model", exist_ok=True)

# Upload the model files
api.upload_folder(
     folder_path="./flan_t5_base_finetuned_MTS_bio_unique_dialogue",
     repo_id=model_name,
     repo_type="model"
 )

CommitInfo(commit_url='https://huggingface.co/hankym/flan_t5_base_finetuned_MTS_bio_unique_dialogue/commit/d67b0cd6bb41278df3e1034a106a7a83a91a0f90', commit_message='Upload folder using huggingface_hub', commit_description='', oid='d67b0cd6bb41278df3e1034a106a7a83a91a0f90', pr_url=None, pr_revision=None, pr_num=None)

## Model evaluation

In [ ]:
# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("hankym/flan_t5_base_finetuned_MTS_bio_unique_dialogue")
model = AutoModelForSeq2SeqLM.from_pretrained("hankym/flan_t5_base_finetuned_MTS_bio_unique_dialogue", torch_dtype=torch.bfloat16)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Paths to the files
path_test_short = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Data/Clean_Short_NER_Bio/test_short_bio_NER.csv'

# Load the datasets
test_short = pd.read_csv(path_test_short)

# Check the shape
print("Shape of short_dialogue_test_data:")
print(test_short.shape)


Shape of short_dialogue_test_data:
(222, 7)


In [ ]:
sub_set_test = test_short[:5]

In [ ]:
sub_set_test = Dataset.from_pandas(sub_set_test)

In [ ]:
test_short = Dataset.from_pandas(test_short)

In [ ]:
# tokenize
max_input_length = 1024
max_output_length = 512
def tokenize_function(batch):
    start_prompt = (
    "Summarize the following patient-doctor. "
    "Include all medically relevant information, including family history, "
    "diagnosis, past medical and surgical history, immunizations, lab results, and known allergies. "
    "Use the following medical and chemical entities extracted from the dialogue to help summarization, but do not overly use them. "
    "Entities:\n\n")

    # Define the end of the prompt to indicate where the summary should go
    end_prompt = '\n\nSummary: '

    # Construct the full prompt for each dialogue in the batch, using the corresponding entities
    prompt = [start_prompt + entities + "\n\n" + 'Dialogue: \n\n' + dialogue + end_prompt
               for entities, dialogue in zip(batch['bio_ner_unique_no_label'], batch['dialogue'])]

    # tokenize the inputs and labels
    inputs = tokenizer(
        prompt,
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["section_text"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
sub_set_test = sub_set_test.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text","dialogue","bio_ner_label", "bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
test_short = test_short.map(
    tokenize_function,
    batched=True,
    batch_size=batch_size,
    remove_columns=["section_header", "section_text","dialogue","bio_ner_label", "bio_ner_unique_label","bio_ner_no_label","bio_ner_unique_no_label"],
)

Map:   0%|          | 0/222 [00:00<?, ? examples/s]

In [ ]:
def model_generation(model, tokenizer, dataset):
    model.eval()  # Ensure the model is in evaluation mode
    dialogues = []
    predictions = []
    references = []

    for example in dataset:
        input_ids = torch.tensor([example["input_ids"]], dtype=torch.long)
        reference_ids = example["labels"]

        # Extract dialogue
        dialogue_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        dialogues.append(dialogue_text)

        # Generate summary
        outputs = model.generate(input_ids)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Since reference IDs might contain -100, filter those out before decoding
        valid_reference_ids = [rid for rid in reference_ids if rid != -100]
        reference_text = tokenizer.decode(valid_reference_ids, skip_special_tokens=True)

        # Append to lists for ROUGE calculation
        predictions.append(generated_text)
        references.append(reference_text)

    return dialogues, predictions, references


In [ ]:
# Call the function with the correct argument
dialogues, t5_predictions, t5_references = model_generation(model, tokenizer, test_short)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(t5_predictions, t5_references):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.29181198432819555
Average ROUGE-2: 0.12510665852548494
Average ROUGE-L: 0.25267968101398663


In [ ]:
with torch.cuda.amp.autocast():
    dialogues, t5_predictions, t5_references = model_generation(model, tokenizer, test_short)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores for each pair of prediction and reference
rouge_scores = []
for pred, ref in zip(t5_predictions, t5_references):
    score = scorer.score(pred, ref)
    rouge_scores.append(score)


# Calculate average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print average ROUGE scores
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1: 0.29181198432819555
Average ROUGE-2: 0.12510665852548494
Average ROUGE-L: 0.25267968101398663


In [ ]:
data = {
    "Dialogue": dialogues,
    "Reference": t5_references,
    "Prediction": t5_predictions,
}
df = pd.DataFrame(data)

import io
import os

# Specify the directory to save the CSV files
output_dir = '/content/drive/MyDrive/Colab Notebooks/w266_Project Ideas/Results/Models/Bio'

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)
df.to_csv(os.path.join(output_dir,"flan_t5_base_NER_bio_predictions.csv"), index=False)

In [ ]:
print(t5_predictions)

['The patient is a 40-year-old male who lives in the same household as the', 'No cancer. No rashes, changes in moles, or weight loss.', 'Non-primary care doctor.', 'The patient is a 6 year old male who is a follow up for his ADHD medications', 'The patient has had an abortion recently. She is still bleeding. She will see our Gyn', 'Atrial fibrillation. No dizziness.', 'The patient has no sugar or BP problem. The patient has no thyroid or other hormone related', 'The patient is a 63-year-old male with a history of kidney stones', 'The patient is a 50-year-old female who has had a fall and ', 'No swelling or mass. No pain.', 'Dislocated wrist when he was a kid.', 'Not known.', 'None.', 'Disseminated CMV infection. Polymyositis. He is on', 'The patient is a mall-bound patient. She walks around at the mall to get her', 'The patient has a swollen lymph node for 2 days.', 'No previous medical issues. No previous surgery or hospitalization.', 'The patient is a 69-year-old male who passed out 